In [ ]:
ALTER SESSION SET TIMEZONE = 'Europe/London';

In [ ]:
-- CREATE OR REPLACE TABLE PROD.SILVER.ADO_PROJECTS (
--     ID STRING,
--     URL STRING,
--     NAME STRING,
--     STATE STRING,
--     REVISION STRING,
--     VISIBILITY STRING,
--     DESCRIPTION STRING,
--     LASTUPDATETIME DATETIME,
--     TIMESTAMP TIMESTAMP_TZ
-- );
TRUNCATE TABLE PROD.SILVER.ADO_PROJECTS;

INSERT INTO PROD.SILVER.ADO_PROJECTS
SELECT
    ID,
    URL,
    NAME,
    STATE,
    REVISION,
    VISIBILITY,
    DESCRIPTION,
    CAST(LASTUPDATETIME AS DATETIME) AS LASTUPDATETIME,
    CURRENT_TIMESTAMP AS TIMESTAMP
FROM PROD.RAW.ADO_PROJECTS;



In [ ]:
-- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # --

-- CREATE OR REPLACE TABLE PROD.SILVER.ADO_DEFINITIONS(
--     ID FLOAT,
--     URL VARCHAR,
--     NAME VARCHAR,
--     PATH VARCHAR,
--     TYPE VARCHAR,
--     REVISION FLOAT,
--     CREATEDDATE TIMESTAMP_TZ,
--     QUEUESTATUS VARCHAR,
--     PROJECTID VARCHAR,
--     PROJECT VARCHAR,
--     TIMESTAMP TIMESTAMP_TZ
-- );

TRUNCATE TABLE PROD.SILVER.ADO_DEFINITIONS;

INSERT INTO PROD.SILVER.ADO_DEFINITIONS
SELECT
    ID,
    URL,
    NAME,
    PATH,
    TYPE,
    REVISION,
    CREATEDDATE,
    QUEUESTATUS,
    PROJECTID,
    PROJECTNAME,
    CURRENT_TIMESTAMP() AS TIMESTAMP
FROM PROD.RAW.ADO_DEFINITIONS;



In [ ]:
-- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # --

-- CREATE OR REPLACE TABLE PROD.SILVER.ADO_PIPELINES(
--     ID FLOAT,
--     URL VARCHAR,
--     NAME VARCHAR,
--     FOLDER VARCHAR,
--     REVISION FLOAT,
--     TIMESTAMP TIMESTAMP_TZ
-- );
TRUNCATE TABLE PROD.SILVER.ADO_PIPELINES;

INSERT INTO PROD.SILVER.ADO_PIPELINES
SELECT
    ID,
    URL,
    NAME,
    FOLDER,
    REVISION,
    CURRENT_TIMESTAMP() AS TIMESTAMP
FROM PROD.RAW.ADO_PIPELINES;

In [ ]:


-- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # --

-- CREATE OR REPLACE TABLE PROD.SILVER.ADO_REPOS(
--     ID VARCHAR,
--     URL VARCHAR,
--     NAME VARCHAR,
--     SIZE FLOAT,
--     ISDISABLED BOOLEAN,
--     DEFAULTBRANCH VARCHAR,
--     ISINMAINTENANCE BOOLEAN,
--     PROJECT VARCHAR,
--     TIMESTAMP TIMESTAMP_TZ
-- );
TRUNCATE TABLE PROD.SILVER.ADO_REPOS;

INSERT INTO PROD.SILVER.ADO_REPOS
SELECT
    REPO_ID as ID,
    URL,
    NAME,
    SIZE,
    ISDISABLED,
    DEFAULTBRANCH,
    ISINMAINTENANCE,
    PROJECT,
    CURRENT_TIMESTAMP() AS TIMESTAMP
FROM PROD.RAW.ADO_REPOS;



In [ ]:

-- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # --

-- CREATE OR REPLACE TABLE PROD.SILVER.ADO_AGED_PULL_REQUESTS (
--     ONEDAYTICKETS INT,
--     THREEDDYTICKETS INT,
--     SEVENDAYTICKETS INT,
--     OLDERTICKETS INT,
--     TIMESTAMP TIMESTAMP_TZ
-- );

TRUNCATE TABLE PROD.SILVER.ADO_AGED_PULL_REQUESTS;

INSERT INTO PROD.SILVER.ADO_AGED_PULL_REQUESTS
SELECT
    SUM(CASE WHEN DATEDIFF(DAY, TO_TIMESTAMP_TZ(REGEXP_REPLACE(CREATIONDATE, '(\\d{2})(\\d{2})$', '\\1:\\2')), CURRENT_TIMESTAMP()) < 1 THEN 1 ELSE 0 END) AS ONEDAYTICKETS,
    SUM(CASE WHEN DATEDIFF(DAY, TO_TIMESTAMP_TZ(REGEXP_REPLACE(CREATIONDATE, '(\\D{2})(\\D{2})$', '\\1:\\2')), CURRENT_TIMESTAMP()) BETWEEN 1 AND 3 THEN 1 ELSE 0 END) AS THREEDDYTICKETS,
    SUM(CASE WHEN DATEDIFF(DAY, TO_TIMESTAMP_TZ(REGEXP_REPLACE(CREATIONDATE, '(\\D{2})(\\D{2})$', '\\1:\\2')), CURRENT_TIMESTAMP()) BETWEEN 4 AND 7 THEN 1 ELSE 0 END) AS SEVENDAYTICKETS,
    SUM(CASE WHEN DATEDIFF(DAY, TO_TIMESTAMP_TZ(REGEXP_REPLACE(CREATIONDATE, '(\\D{2})(\\D{2})$', '\\1:\\2')), CURRENT_TIMESTAMP()) > 7 THEN 1 ELSE 0 END) AS OLDERTICKETS,
    CURRENT_TIMESTAMP() AS TIMESTAMP
FROM PROD.RAW.ADO_PULL_REQUESTS_SNOWFLAKE
WHERE STATUS = 'active';




In [ ]:
-- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # --

-- CREATE OR REPLACE TABLE PROD.SILVER.ADO_APPROVALS (
--     ID VARCHAR,
--     STATUS VARCHAR,
--     PIPELINEID VARCHAR,
--     PIPELINENAME VARCHAR,
--     PIPELINEOWNERID VARCHAR,
--     CREATEDON TIMESTAMP_TZ,
--     MINREQUIREDAPPROVERS FLOAT,
--     TIMESTAMP TIMESTAMP_TZ
-- );

TRUNCATE TABLE PROD.SILVER.ADO_APPROVALS;

INSERT INTO PROD.SILVER.ADO_APPROVALS
SELECT
    ID,
    STATUS,
    PIPELINEID,
    PIPELINENAME,
    PIPELINEOWNERID,
    CAST(CREATEDON AS DATETIME) AS CREATEDON,
    MINREQUIREDAPPROVERS,
    CURRENT_TIMESTAMP AS TIMESTAMP
FROM PROD.RAW.ADO_APPROVALS;


In [ ]:

-- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # --

-- CREATE OR REPLACE TABLE prod.silver.ADO_BRANCHES
-- (
-- PROJECT_NAME string,
-- REPO_ID string,
-- BRANCH_NAME string
-- );

truncate prod.silver.ADO_BRANCHES;

insert into prod.silver.ADO_BRANCHES 
-- create or replace table prod.silver.ADO_BRANCHES as
  select  PROJECT_NAME::TEXT AS PROJECT_NAME,
    REPO_ID::TEXT AS REPO_ID,
    BRANCH_NAME::TEXT AS BRANCH_NAME
FROM prod.raw.ado_branches;



In [ ]:
-- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # ---- # --

-- CREATE OR REPLACE TABLE prod.silver.ADO_COMMITS (
--     REPO_ID TEXT,
--     BRANCH_NAME TEXT,
--     COMMIT_ID TEXT,
--     AUTHOR_EMAIL TEXT,
--     DATE DATETIME,
--     MESSAGE TEXT,
--     TIMESTAMP TIMESTAMP_LTZ
-- );

truncate table prod.silver.ADO_COMMITS;

INSERT INTO prod.silver.ADO_COMMITS (
    REPO_ID,
    BRANCH_NAME,
    COMMIT_ID,
    AUTHOR_EMAIL,
    DATE,
    MESSAGE,
    TIMESTAMP
)
SELECT 
    REPO_ID::TEXT,
    BRANCH_NAME::TEXT,
    VALUE:commitId::TEXT,
    VALUE:author:email::TEXT,
    VALUE:author:date::DATETIME,
    VALUE:comment::TEXT,
    CURRENT_TIMESTAMP()
FROM prod.raw.ado_commits,

LATERAL FLATTEN(INPUT => PARSE_JSON(RESULT_COMMIT):value);
